## 텐서플로우 DatasetAPI

대규모 데이터를 다루기 위해서 이전 데이터 형태들을 벗어날 필요가 있습니다.
(속도면에서도 이점이 큽니다)

### Dataset 구조

In [4]:
import keras
from zipfile import ZipFile
import pandas as pd
import io
with ZipFile("../Data/고객 대출등급 분류 해커톤.zip","r") as f:
    data=io.BytesIO(f.read("고객 대출등급 분류 해커톤/train.csv"))
from sklearn import model_selection as mod
import numpy as np
import tensorflow as tf

data=pd.read_csv(data)

#### Tensor

기본적으로 텐서를 살펴봅시다

기본적으로 아래의 혼합 타입을 numpy 에선 가능했습니다.

하지만 Tensorflow.Tensor 에서 Mixed Type 을 지원하지 않습니다.

(이후 사용할 Tensorflow.Dataset API 를 사용할 때 기억해야합니다)

In [13]:
np.array([1,2,3,"a","b","c"])

array(['1', '2', '3', 'a', 'b', 'c'], dtype='<U21')

In [16]:
try:
    tf.constant([1,2,3,"a","b","c"])
except:
    print("Mixed Type Error")

Mixed Type Error


따라서 이전에 사용하던 다음과 같은 데이터를 하나의 텐서로 담는 것은 불가능합니다.

In [18]:
print(data.loc[0])

ID              TRAIN_00000
대출금액               12480000
대출기간              36 months
근로기간                6 years
주택소유상태                 RENT
연간소득               72000000
부채_대비_소득_비율            18.9
총계좌수                     15
대출목적                  부채 통합
최근_2년간_연체_횟수              0
총상환원금                     0
총상환이자                   0.0
총연체금액                   0.0
연체계좌수                   0.0
대출등급                      C
Name: 0, dtype: object


하지만 아래처럼 단일 타입은 가능합니다.
(간단히 설명하면 int 와 float 은 float으로 통일됩니다)

In [21]:
tf.constant(data.select_dtypes("object").loc[0])

<tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'TRAIN_00000', b' 36 months', b'6 years', b'RENT',
       b'\xeb\xb6\x80\xec\xb1\x84 \xed\x86\xb5\xed\x95\xa9', b'C'],
      dtype=object)>

string 타입은 바이트 타입으로 변환됩니다.

In [26]:
text="RENT"
print(type(text))
text2=b"RENT"
print(type(text2))

<class 'str'>
<class 'bytes'>


단순히 파이썬을 사용하면서 바이트 타입을 쓸 일이 얼마 없었다.

바이트 타입은 우리가 쓰는 문자가 아닌 이진데이터로 이루어져있다.

인덱스 슬라이싱을 하면 무슨 말인지 바로 알 수 있다

In [29]:
b"RENT"[0]

82

이후 다룰 BERT Tokenizer를 사용할 때 이진데이터를 제공해 문제가 생김을 고려해야합니다

### DataSet Making

여러가지 방법으로 데이터셋을 만들어보겠습니다.

__generater__

이는 제너레이터를 통해 만드는 것(쓰는 일이 거의 없지만 소개하겠습니다)

제너레이터를 만들어 확인해보죠.(제너레이터 설명은 생략합니다.)

In [50]:
use_col=["대출금액","연간소득"]
def read_df():
    i=0
    while True:
        yield data.loc[i,use_col]

In [51]:
tf_data=tf.data.Dataset.from_generator(read_df,output_signature=tf.TensorSpec(shape=(2,),dtype=tf.int32))

In [52]:
for i in tf_data.take(1):
    print(i)

tf.Tensor([12480000 72000000], shape=(2,), dtype=int32)


2025-01-14 14:18:53.285463: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


장점은 파이썬을 활용한 제너레이터 함수의 자유도를 사용할 수 있다는 점입니다.

__tensor_slices__

가장 많이 사용할 방법입니다.(그만큼 편합니다)

다양한 형태를 자동으로 slicing 합니다

In [59]:
print(data[use_col].values[:3])

[[ 12480000  72000000]
 [ 14400000 130800000]
 [ 12000000  96000000]]


In [60]:
tf_data=tf.data.Dataset.from_tensor_slices(data[use_col].values)
for i in tf_data.take(3):
    print(i)

tf.Tensor([12480000 72000000], shape=(2,), dtype=int64)
tf.Tensor([ 14400000 130800000], shape=(2,), dtype=int64)
tf.Tensor([12000000 96000000], shape=(2,), dtype=int64)


2025-01-14 14:26:43.441535: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


딕셔너리 형태도 슬라이싱합니다.

In [76]:
data_dc={key:data.loc[:,key].values.tolist() for key in use_col}

In [77]:
tf_data=tf.data.Dataset.from_tensor_slices(data_dc)

In [78]:
for i in tf_data.take(3):
    print(i)

{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}


2025-01-14 14:42:37.292417: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Dataset method

Tensorflow Dataset API는 스트리밍 API 입니다. 따라서  인덱스 슬라이싱 등에 유용한 형태로 존재하지 않습니다.

따라서 아래의 메소드들을 알아두면 좋습니다.

또한 모든 메소드는 새로운 데이터셋을 만들기 떄문에 반환받아야합니다.
(강조표시는 꼭 알아두면 좋습니다)

take : 특정개수까지 스트리밍

In [82]:
for i in tf_data.take(5):
    print(i)

{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=132000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=18000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=71736000>}


2025-01-14 14:46:04.805048: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


skip :  특정개수를 지나침.

In [83]:
for i in tf_data.skip(2).take(3):
    print(i)

{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=132000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=18000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=71736000>}


2025-01-14 14:51:53.464374: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


take_while : 조건이 만족하는 동안 스트리밍

In [87]:
for i in tf_data.take_while(lambda tensor :  tensor["대출금액"] <18_000_000):
    print(i)

{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=132000000>}


2025-01-14 14:53:37.803095: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


__map__ : 사용자 정의 변환을 적용할 수 있습니다(텐서플로우 함수로 변환 가능하다면)

In [91]:
for i in tf_data.map(lambda tensor : (tensor["대출금액"],tensor["연간소득"])).take(5):
    print(i)

(<tf.Tensor: shape=(), dtype=int32, numpy=12480000>, <tf.Tensor: shape=(), dtype=int64, numpy=72000000>)
(<tf.Tensor: shape=(), dtype=int32, numpy=14400000>, <tf.Tensor: shape=(), dtype=int64, numpy=130800000>)
(<tf.Tensor: shape=(), dtype=int32, numpy=12000000>, <tf.Tensor: shape=(), dtype=int64, numpy=96000000>)
(<tf.Tensor: shape=(), dtype=int32, numpy=14400000>, <tf.Tensor: shape=(), dtype=int64, numpy=132000000>)
(<tf.Tensor: shape=(), dtype=int32, numpy=18000000>, <tf.Tensor: shape=(), dtype=int64, numpy=71736000>)


2025-01-14 14:59:21.441646: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [97]:
for i in tf_data.map(lambda tensor : tf.cast(tensor["대출금액"],dtype=tf.int64),num_parallel_calls=2).take(5):
    print(i)

tf.Tensor(12480000, shape=(), dtype=int64)
tf.Tensor(14400000, shape=(), dtype=int64)
tf.Tensor(12000000, shape=(), dtype=int64)
tf.Tensor(14400000, shape=(), dtype=int64)
tf.Tensor(18000000, shape=(), dtype=int64)


2025-01-14 15:14:55.632833: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


__shuffle__ 

버퍼  :  일종의 중간 임시 저장소

Tensorflow DataSet API 는 __스트리밍__API 이기때문에 대규모 데이터를 변환하는것도 가능합니다.

하지만 메모리가 부족한 상황에서 이 데이터를 섞는 것은 굉장히 어려운 일입니다.(numpy로는 더더욱 어렵겠죠)

따라서 Tensorflow DataSet 은 버퍼를 다음과 같이 사용합니다.

- 1. 원본 데이터에서 버퍼의 개수만큼 데이터를 스트리밍해 버퍼를 채웁니다
- 2. 버퍼에서 shuffling 을 진행합니다.
- 3. 최종 스트리밍은 버퍼에서 진행되며, 스트리밍과 동시에 데이터를 버퍼에 추가하고 2단계를 실행합니다

여기서 주의할 점은 버퍼의 사이즈가 작다면, 원본 데이터의 마지막 데이터는 똑같은 순서대로(마지막에 가깝게) 뽑힐 리스크가 커집니다.

즉, 완벽한 셔플링은 원본 데이터와 같은 사이즈의 버퍼사이즈를 가진 셔플링입니다

__repeat__ : 데이터셋을 반복합니다.

- 단순반복

In [99]:
for i in tf_data.take(3).repeat(2):
    print(i)

{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}


2025-01-14 15:22:15.768290: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


* 셔플링 후 반복

In [101]:
for i in tf_data.take(3).shuffle(buffer_size=3).repeat(2):
    print(i)

{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}


2025-01-14 15:23:31.888923: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


* 반복마다 셔플링 실시

In [102]:
for i in tf_data.take(3).shuffle(buffer_size=3,reshuffle_each_iteration=True).repeat(2):
    print(i)

{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}


2025-01-14 15:24:34.938512: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Prefetch

이번장의 목표입니다.
학습 속도를 높이는데 중요한 Tensorflow DataSet의 주요 메소드입입니다.

이 메소드는 훈련동안 미리 다음훈련할 배치 데이터를 CPU를 활용해 준비합니다.

In [103]:
for i in tf_data.take(3).shuffle(buffer_size=3,reshuffle_each_iteration=True).repeat(3).prefetch(1):
    print(i)

{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12000000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=96000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=12480000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=72000000>}
{'대출금액': <tf.Tensor: shape=(), dtype=int32, numpy=14400000>, '연간소득': <tf.Tensor: shape=(), dtype=int64, numpy=130800000>}
{'대출금액': <tf.Tensor: shape=()

2025-01-14 15:34:30.186214: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


일반적으로 1개의 배치만 준비해도 충분하지만, 그렇지 않을 경우를 고려하고 싶다면, `tf.data.AUTOTUNE`을 전달하면 됩니다

그 외의 메소드는 차후 사용할 환경에서 소개하겠습니다.